In [36]:
import pandas as pd
import re
from imblearn.over_sampling import ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [3]:
# Import data
train = pd.read_csv("../data/train_complete.csv")
test = pd.read_csv("../data/test_complete.csv")

In [14]:
test["P1_c_Amarela(OneHot)"]

0      False
1       True
2      False
3      False
4      False
       ...  
946    False
947    False
948    False
949    False
950    False
Name: P1_c_Amarela(OneHot), Length: 951, dtype: bool

## Resampling

In [5]:
# Dataprep - toda transformação deve ser criada sobre o teste e somente aplicada sobre o treino
def apply_random_oversampling(df, oversample_col='', exclude_category=None):
    '''
    df: dataframe with all columns
    oversample_col: column to apply the oversampling
    exclude_category: list of categories on oversample_col to set apart while oversampling
    '''
    df_cat = pd.DataFrame(columns = df.columns)
    df_ = df.copy()
    
    # Separando categorias específica se necessário
    if exclude_category is not None:
        
        for cat in exclude_category:
            
            df_cat = pd.concat([df_cat, df[df[oversample_col]==cat]]).reset_index(drop=True)
            
            # removendo categorias que não serão oversampled
            df_ = df_[df[oversample_col]!=cat].reset_index()
    
    # Separando coluna de oversampling
    X = df_.drop(columns=[oversample_col])
    y = df_[oversample_col]

    # Aplicando Random Oversampling
    oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X, y)

    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    df_resampled[oversample_col] = y_resampled

    # Exibir o DataFrame balanceado
    print(df_resampled[oversample_col].value_counts())
    
    # inserindo de volta as categorias que foram separadas
    if not df_cat.empty:
        df_resampled = pd.concat([df_resampled, df_cat]).reset_index(drop=True)
        
    return df_resampled

In [6]:
def apply_random_undersampling(df, undersample_col='', strategy={}):
    '''
    df: dataframe with all columns
    undersample_col: column to apply the undersampling
    srtategy: dictionary with the category as key and number of samples requested as value
    '''

    X = df.drop(columns=[undersample_col])
    y = df[undersample_col]

    # aplicando undersampling
    undersampler = RandomUnderSampler(sampling_strategy=strategy, random_state=42)
    X_resampled, y_resampled = undersampler.fit_resample(X, y)

    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    df_resampled[undersample_col] = y_resampled

    # Exibir o DataFrame balanceado e a distribuição de classes
    print(df_resampled[undersample_col].value_counts())
    
    return df_resampled

In [7]:
col_raca = 'P1_c'
col_genero = 'P1_b'
col_idade ='P1_a_1'
col_regiao = 'P1_i_2'

df_train_resampled = train.copy()

# cols_resampling = [col_raca, col_genero, col_idade ] # , col_regiao]


# for col in cols_resampling:
#     df_train_resampled = apply_random_oversampling(df_train_resampled, oversample_col=col)

In [8]:
onehot = ["P1_m", "P3_c","P4_a", "P4_a2", "P4_b", "P4_c", "P4_d", "P4_e", "P4_g", "P4_j", "P5_b", "P6_a", "P6_b", "P6_g", "P6_h", "P7_a", "P7_b", "P7_d", "P8_a", "P8_b", "P8_c", "P8_d", "P2_b", "P2_d", "P2_f", "P2_o", "P2_r", "P6_c", "P6_d", "P6_e", "P6_f"]
ordered = ["P1_l", "P2_e", "P2_g", "P2_i", "P2_j", "P2_h"]
sensible = ['P1_c', 'P1_b','P1_a_1','P1_i_2', 'P1_i_1']

# df_feat_selected = df_resampled[ [x +endswith("(OneHot)") for x in onehot ] + [x +endswith("(OrdEnc)") for x in ordered] ]


In [15]:
onehot_cols = [col for col in df_train_resampled.columns if any(col.startswith(c) for c in onehot + sensible) and (col.endswith('(OneHot)') or col.endswith('(FromList)'))]
ordered_cols = [col for col in df_train_resampled.columns if any(col.startswith(c) for c in ordered) and col.endswith('(OrdEnc)')]
df_train_selected = df_train_resampled[onehot_cols+ordered_cols]

len(ordered_cols)

6

In [16]:
df_train_selected[[col for col in df_train_selected.columns if "P1_c" in col]].head(1)

,P1_c_Amarela(OneHot),P1_c_Branca(OneHot),P1_c_Indígena(OneHot),P1_c_Outra(OneHot),P1_c_Parda(OneHot),P1_c_Prefiro não informar(OneHot),P1_c_Preta(OneHot)
0,False,True,False,False,False,False,False


In [17]:
# for sens_col in sensible:
#     df_feat_selected = pd.concat([df_feat_selected, pd.get_dummies(df_feat_selected[sens_col])], axis=1)

df_train_selected.columns

Index(['P2_o_1(FromList)', 'P2_o_2(FromList)', 'P2_o_3(FromList)',
       'P2_o_4(FromList)', 'P2_o_5(FromList)', 'P2_o_6(FromList)',
       'P2_o_7(FromList)', 'P2_o_8(FromList)', 'P2_o_9(FromList)',
       'P2_o_10(FromList)',
       ...
       'P6_g_Postgres/MySQL(OneHot)', 'P6_g_Presto(OneHot)',
       'P6_g_Snowflake(OneHot)', 'P6_g_Teradata(OneHot)', 'P1_l(OrdEnc)',
       'P2_e(OrdEnc)', 'P2_g(OrdEnc)', 'P2_i(OrdEnc)', 'P2_j(OrdEnc)',
       'P2_h(OrdEnc)'],
      dtype='object', length=335)

In [18]:
test_feat_selected = test[onehot_cols+ordered_cols]

# for sens_col in sensible:
#     test_feat_selected = pd.concat([test_feat_selected, pd.get_dummies(test_feat_selected[sens_col])], axis=1)

In [19]:
X_train = df_train_selected.drop(columns=['P2_h(OrdEnc)'])
y_train = df_train_selected['P2_h(OrdEnc)']

X_test = test_feat_selected.drop(columns=['P2_h(OrdEnc)'])
y_test = test_feat_selected['P2_h(OrdEnc)']

# Teste
# gambiarra porque no teste não tinha nenhuma classe indígena, então a coluna não foi criada no get dummies
# X_test['Indígena'] = 0

# Preserve order
X_test = X_test[X_train.columns]

In [20]:
print(len(X_train.columns))
print(len(X_test.columns))

X_train.columns

334
334


Index(['P2_o_1(FromList)', 'P2_o_2(FromList)', 'P2_o_3(FromList)',
       'P2_o_4(FromList)', 'P2_o_5(FromList)', 'P2_o_6(FromList)',
       'P2_o_7(FromList)', 'P2_o_8(FromList)', 'P2_o_9(FromList)',
       'P2_o_10(FromList)',
       ...
       'P6_g_Oracle(OneHot)', 'P6_g_Postgres/MySQL(OneHot)',
       'P6_g_Presto(OneHot)', 'P6_g_Snowflake(OneHot)',
       'P6_g_Teradata(OneHot)', 'P1_l(OrdEnc)', 'P2_e(OrdEnc)', 'P2_g(OrdEnc)',
       'P2_i(OrdEnc)', 'P2_j(OrdEnc)'],
      dtype='object', length=334)

In [21]:
# See if order of train and test columns are exactly the same
assert (X_train.columns == X_test.columns).all()

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import joblib

In [23]:
X_train[[col for col in X_train.columns if "P1_c" in col]]

,P1_c_Amarela(OneHot),P1_c_Branca(OneHot),P1_c_Indígena(OneHot),P1_c_Outra(OneHot),P1_c_Parda(OneHot),P1_c_Prefiro não informar(OneHot),P1_c_Preta(OneHot)
0,False,True,False,False,False,False,False
1,False,False,False,False,False,False,True
2,False,True,False,False,False,False,False
3,False,True,False,False,False,False,False
4,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...
3796,False,True,False,False,False,False,False
3797,False,True,False,False,False,False,False
3798,False,True,False,False,False,False,False
3799,False,True,False,False,False,False,False


In [24]:
# Logistic Regression
logistic_regression = LogisticRegression(max_iter=100)
logistic_regression.fit(X_train, y_train)

c:\Users\vi_ma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
# KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [26]:
# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [27]:
# Fit the QDA model with the balanced dataset
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

c:\Users\vi_ma\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [29]:
set(X_train.columns)-set(X_test.columns)

set()

In [30]:
X_train.columns

Index(['P2_o_1(FromList)', 'P2_o_2(FromList)', 'P2_o_3(FromList)',
       'P2_o_4(FromList)', 'P2_o_5(FromList)', 'P2_o_6(FromList)',
       'P2_o_7(FromList)', 'P2_o_8(FromList)', 'P2_o_9(FromList)',
       'P2_o_10(FromList)',
       ...
       'P6_g_Oracle(OneHot)', 'P6_g_Postgres/MySQL(OneHot)',
       'P6_g_Presto(OneHot)', 'P6_g_Snowflake(OneHot)',
       'P6_g_Teradata(OneHot)', 'P1_l(OrdEnc)', 'P2_e(OrdEnc)', 'P2_g(OrdEnc)',
       'P2_i(OrdEnc)', 'P2_j(OrdEnc)'],
      dtype='object', length=334)

In [31]:
X_test.columns

Index(['P2_o_1(FromList)', 'P2_o_2(FromList)', 'P2_o_3(FromList)',
       'P2_o_4(FromList)', 'P2_o_5(FromList)', 'P2_o_6(FromList)',
       'P2_o_7(FromList)', 'P2_o_8(FromList)', 'P2_o_9(FromList)',
       'P2_o_10(FromList)',
       ...
       'P6_g_Oracle(OneHot)', 'P6_g_Postgres/MySQL(OneHot)',
       'P6_g_Presto(OneHot)', 'P6_g_Snowflake(OneHot)',
       'P6_g_Teradata(OneHot)', 'P1_l(OrdEnc)', 'P2_e(OrdEnc)', 'P2_g(OrdEnc)',
       'P2_i(OrdEnc)', 'P2_j(OrdEnc)'],
      dtype='object', length=334)

In [32]:
cols_when_model_fited = logistic_regression.feature_names_in_

set(cols_when_model_fited) - set(X_test[X_train.columns])

set()

In [37]:
import numpy as np
models = {
    'QDA': qda,
    'Logistic Regression': logistic_regression,
    'KNN': knn,
    'Decision Tree': decision_tree
}

cols_when_model_fited = logistic_regression.feature_names_in_

results = []
for model_name, model in models.items():
    #X_test_aligned = X_test[X_train.columns]
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = np.mean(precision_score(y_test, y_pred, average=None, zero_division=0))
    recall = np.mean(recall_score(y_test, y_pred, average=None, zero_division=0))
    f1 = np.mean(f1_score(y_test, y_pred, average=None, zero_division=0))
    results.append([model_name, accuracy, precision, recall, f1])

# Store the results in a DataFrame
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Print the results
results_df

,Model,Accuracy,Precision,Recall,F1-Score
0,QDA,0.201893,0.068899,0.111761,0.068859
1,Logistic Regression,0.302839,0.229621,0.223690,0.225422
2,KNN,0.277603,0.261929,0.215394,0.225214
3,Decision Tree,0.275499,0.203945,0.206385,0.202931


In [38]:
# Define sens_col
sens_col = 'P1_b_Feminino(OneHot)'

# Use proper boolean indexing
test = X_test[X_test[sens_col] == True]

test[sens_col]


3      True
8      True
10     True
22     True
24     True
       ... 
927    True
930    True
932    True
935    True
938    True
Name: P1_b_Feminino(OneHot), Length: 213, dtype: bool

In [41]:
# Teste POR CLASSE SENSÍVEL
model = logistic_regression

for sens in sensible:
    print(f"Feature {sens}")
    onehot_sensible_cols = [col for col in df_train_resampled.columns if col.startswith(sens) and (col.endswith('(OneHot)') or col.endswith('(FromList)'))]
    results_sensible = []
    for sens_col in onehot_sensible_cols:
        X_test_filtered = X_test[X_test[sens_col] == True]
        y_test_filtered = y_test[X_test[sens_col] == True]
        if X_test_filtered.shape[0]>0:
            y_pred = model.predict(X_test_filtered)
            accuracy = accuracy_score(y_test_filtered, y_pred)
            f1 = f1_score(y_test_filtered, y_pred, average='macro', zero_division=0)
            # print(f"Classe {sens_col}: acc={accuracy} / f1={f1}\n")
            results_sensible.append([sens_col, accuracy, f1])

    df_results_sensible = pd.DataFrame(results_sensible, columns=['column','accuracy','f1'])
    display(df_results_sensible.head(10).sort_values('f1', ascending=False))
    display(df_results_sensible.tail(10).sort_values('f1', ascending=False))

,column,accuracy,f1
0,P1_a_1_17-21(OneHot),0.181818,0.042328
7,P1_a_1_50-54(OneHot),0.136364,0.104762
1,P1_a_1_22-24(OneHot),0.225000,0.146699
3,P1_a_1_30-34(OneHot),0.303371,0.168205
4,P1_a_1_35-39(OneHot),0.330827,0.183102
9,P1_b_Feminino(OneHot),0.281690,0.196317
6,P1_a_1_45-49(OneHot),0.322581,0.200000
5,P1_a_1_40-44(OneHot),0.279070,0.239285
2,P1_a_1_25-29(OneHot),0.331126,0.248706
8,P1_a_1_55+(OneHot),0.500000,0.458333


,column,accuracy,f1
47,P1_i_2_Não estou em nenhuma região brasileira(...,0.222222,0.161376
41,P1_i_1_SC(OneHot),0.291667,0.172495
49,P1_i_2_Sul(OneHot),0.318408,0.181209
46,P1_i_2_Norte(OneHot),0.200000,0.187500
48,P1_i_2_Sudeste(OneHot),0.295775,0.222933
40,P1_i_1_RS(OneHot),0.366667,0.223063
45,P1_i_2_Nordeste(OneHot),0.360465,0.224698
43,P1_i_1_SP(OneHot),0.321337,0.235126
44,P1_i_2_Centro-oeste(OneHot),0.285714,0.241204
42,P1_i_1_SE(OneHot),0.600000,0.360000
